# Performing a chemical equilibrium calculation

This tutorial demonstrates how to use Reaktoro to perform a chemical equilibrium calculation.

In [130]:
from reaktoro import *  # import everything from the reaktoro package

In [131]:
db = Database('supcrt98.xml')  # We need a thermodynamic database that enable us to compute thermodynamic properties of species and reactions.

In [132]:
editor = ChemicalEditor(db)  # Let's use ChemicalEditor to indicate the phases, and their species, that may potentially exist at equilibrium

In [133]:
editor.addAqueousPhaseWithElements('H O Na Cl C Ca Mg Si');  # Let's consider an aqueous phase with species that can be formed with given chemical element symbols

In [134]:
editor.addGaseousPhase('H2O(g) CO2(g)');  # Let's consider the possibility of a gaseous phase with the given species names

In [135]:
# Below we consider all possible pure minerals that could exist in our equilibrium calculations
editor.addMineralPhase('Halite');
editor.addMineralPhase('Calcite');
editor.addMineralPhase('Magnesite');
editor.addMineralPhase('Dolomite');
editor.addMineralPhase('Quartz');

In [136]:
system = ChemicalSystem(editor)  # Important step where we create the chemical system with the information so far collected in the ChemicalEditor object `editor`.

In [137]:
problem = EquilibriumProblem(system)  # We use EquilibriumProblem to specify the conditions at which our system should be in equilibrium

In [138]:
# Specify temperature and pressure conditions
problem.setTemperature(70, 'celsius');
problem.setPressure(100, 'bar');

In [139]:
# Specify the initial condition for substance amounts
problem.add('H2O', 1.0, 'kg');
problem.add('CO2', 2.0, 'mol');
problem.add('NaCl', 1.0, 'mol');
problem.add('CaCO3', 10.0, 'g');
problem.add('MgCO3', 5.0, 'g');
problem.add('Quartz', 1.0, 'mol');

In [140]:
state = equilibrate(problem)  # Perform a fast Gibbs energy minimization calculation to calculate the chemical equilibrium state

In [141]:
state.speciesAmounts()  # Access the amounts of species at equilibrium with this method

array([3.47123978e-22, 7.09041358e-01, 2.76400208e-06, 4.50376015e-03,
       2.85240163e-09, 2.85810008e-02, 8.75324984e-06, 3.35861110e-03,
       3.28732970e-04, 4.71317015e-09, 9.12301141e-01, 5.06141409e-21,
       3.07298696e-22, 2.63196950e-22, 1.90499809e-22, 1.63623782e-05,
       4.01736010e-22, 5.54588771e+01, 3.08641376e-21, 7.19042977e-02,
       1.04497165e-06, 9.78124432e-20, 2.57623996e-22, 6.59775082e-22,
       4.89137994e-08, 1.91516163e-04, 1.70161991e-10, 1.17976629e-03,
       1.13625082e-07, 1.50720217e-04, 1.45532386e-09, 9.16468526e-01,
       8.35310163e-02, 4.51871969e-07, 6.02482574e-09, 9.08508427e-16,
       2.30942364e-08, 4.46600066e-04, 1.12491686e-02, 1.25265042e+00,
       2.08357234e-21, 5.35222470e-03, 3.11553045e-21, 5.77800849e-02,
       9.99552896e-01])

In [142]:
# Print the name and the amount (in mol) of each species
for species in system.species():
    print(f'{species.name():>15} = {state.speciesAmount(species.name())}')

         CO(aq) = 3.4712397789883406e-22
        CO2(aq) = 0.7090413580406868
          CO3-- = 2.764002078730986e-06
      Ca(HCO3)+ = 0.004503760152884106
     Ca(HSiO3)+ = 2.852401632270354e-09
           Ca++ = 0.028581000794659285
      CaCO3(aq) = 8.753249841072126e-06
          CaCl+ = 0.003358611101676718
      CaCl2(aq) = 0.00032873296992645386
          CaOH+ = 4.71317014865048e-09
            Cl- = 0.9123011414436385
           ClO- = 5.0614140939113705e-21
          ClO2- = 3.072986964060557e-22
          ClO3- = 2.631969503968108e-22
          ClO4- = 1.9049980937378452e-22
             H+ = 1.6362378166444628e-05
         H2(aq) = 4.0173600965037235e-22
         H2O(l) = 55.45887713293843
       H2O2(aq) = 3.086413757652217e-21
          HCO3- = 0.07190429769285117
        HCl(aq) = 1.0449716497891676e-06
       HClO(aq) = 9.781244321024934e-20
      HClO2(aq) = 2.5762399645938917e-22
           HO2- = 6.597750822554504e-22
         HSiO3- = 4.891379940477807e-08
      Mg

In [143]:
# You can also output the chemical state to a file (check the results folder on the left pane of jupyter lab!)
state.output('results/equilibrium-basics-state.txt')

In [144]:
properties = ChemicalProperties(system)  # Use ChemicalProperties to obtain all chemical and thermodynamic properties of a chemical system at some state

In [145]:
properties.update(state.temperature(), state.pressure(), state.speciesAmounts())  # Compute the chemical properties of the system at given state

In [146]:
properties = state.properties()  # You can also use this shorter notation, at the expense of creating a new ChemicalProperties object at each call, instead of updating an existing one

In [147]:
# Let's print the activities of each species
lna = properties.lnActivities().val  # Get the values of ln activities of the species (if you use .ddT, .ddP, or .ddn instead of .val, you'll get, respectively, the T, P, n derivatives)
a = numpy.exp(lna)  # Compute the actual activities, not the natural log
for i, species in enumerate(system.species()):
    print(f'{species.name():>15} = {a[i]}')

         CO(aq) = 3.4743439853737896e-22
        CO2(aq) = 0.875504228976846
          CO3-- = 3.874244203328968e-07
      Ca(HCO3)+ = 0.0028560960548854996
     Ca(HSiO3)+ = 1.8088736460931053e-09
           Ca++ = 0.003847759712161054
      CaCO3(aq) = 8.761077561362328e-06
          CaCl+ = 0.0021298904896724312
      CaCl2(aq) = 0.0003290269441400857
          CaOH+ = 2.9888951033381423e-09
            Cl- = 0.5922592416279279
           ClO- = 3.285833083669397e-21
          ClO2- = 1.9949607056141808e-22
          ClO3- = 1.708655389755737e-22
          ClO4- = 1.2275291557377207e-22
             H+ = 1.0401639866084542e-05
         H2(aq) = 4.020952678882997e-22
         H2O(l) = 0.9676752428985484
       H2O2(aq) = 3.089173827801402e-21
          HCO3- = 0.046133412634721226
        HCl(aq) = 1.0459061308029913e-06
       HClO(aq) = 9.789991340249271e-20
      HClO2(aq) = 2.578543804448736e-22
           HO2- = 4.283211673321789e-22
         HSiO3- = 3.1754481524348856e-08
    

In [148]:
# Let's create a pH function that determines the pH of the aqueous solution given the chemical properties of the system (this will be soon simplified!)
evaluate_pH = ChemicalProperty.pH(system)
pH = evaluate_pH(properties)
print(f'The pH of the aqueous phase is {pH.val}.')
print(f'Its sensitivity with respect to speciation, ∂(pH)/∂n, is:')
for i, species in enumerate(system.species()):
    print(f'{species.name():>15} = {pH.ddn[i]}')

The pH of the aqueous phase is 4.982898186792658.
Its sensitivity with respect to speciation, ∂(pH)/∂n, is:
         CO(aq) = 0.007463272238343973
        CO2(aq) = 0.007463272238343973
          CO3-- = 0.01764093171816751
      Ca(HCO3)+ = 0.010007687108299856
     Ca(HSiO3)+ = 0.010007687108299856
           Ca++ = 0.01764093171816751
      CaCO3(aq) = 0.007463272238343973
          CaCl+ = 0.010007687108299856
      CaCl2(aq) = 0.007463272238343973
          CaOH+ = 0.010007687108299856
            Cl- = 0.010007687108299856
           ClO- = 0.010007687108299856
          ClO2- = 0.010007687108299856
          ClO3- = 0.010007687108299856
          ClO4- = 0.010007687108299856
             H+ = -26542.249160597315
         H2(aq) = 0.007463272238343973
         H2O(l) = 0.007370231336413049
       H2O2(aq) = 0.007463272238343973
          HCO3- = 0.010007687108299856
        HCl(aq) = 0.007463272238343973
       HClO(aq) = 0.007463272238343973
      HClO2(aq) = 0.00746327223834397